In [6]:
!pip install -q simpletransformers

     |████████████████████████████████| 204kB 14.0MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 7.4MB 10.5MB/s 
     |████████████████████████████████| 1.5MB 58.1MB/s 
     |████████████████████████████████| 1.8MB 50.9MB/s 
     |████████████████████████████████| 2.9MB 51.1MB/s 
     |████████████████████████████████| 1.1MB 52.8MB/s 
     |████████████████████████████████| 51kB 8.9MB/s 
     |████████████████████████████████| 317kB 53.4MB/s 
     |████████████████████████████████| 4.5MB 51.3MB/s 
     |████████████████████████████████| 112kB 66.7MB/s 
     |████████████████████████████████| 163kB 59.1MB/s 
     |████████████████████████████████| 81kB 12.9MB/s 
     |████████████████████████████████| 890kB 44.7MB/s 
     |████████████████████████████████| 133kB 59.2MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 122kB 57.1MB/s 
     |████████████████████████████████| 71kB 11.6MB/s

In [7]:
!pip install -q wandb

In [4]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import logging
import wandb

In [5]:
#load data
rel_df = pd.read_csv('final_intent_df.csv')
rel_df.head()

,Query,pred
0,what is a book by laura ingalls wilder,book_from_author
1,What did jane austen write?,book_from_author
2,What kind of book is home?,genre
3,what genre is the book in the line of fire: a ...,genre
4,what is the genre of the book the hydrofoil my...,genre


In [8]:
rel_df = rel_df.dropna()

In [9]:
X = rel_df['Query']
Y = rel_df['pred']

In [10]:

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

In [12]:
encoder.classes_

array(['author_of_book', 'book_from_author', 'books_in_genre', 'country',
       'date', 'genre', 'language', 'publisher'], dtype=object)

In [13]:
#split data 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
train_df = pd.DataFrame(zip(x_train, y_train), columns=['text', 'labels'])
test_df = pd.DataFrame(zip(x_test, y_test), columns=['text', 'labels'])

In [16]:
logging.basicConfig(level=logging.INFO)

# Get root logger (all other loggers will be derived from this logger's
# properties)
logger = logging.getLogger()
logger.warning("Is this working?") 

# Get the logger for the huggingface/transformers library.
transformers_logger = logging.getLogger("transformers")

# Set the logging level to warning, meaning display warnings and worse, but 
# don't display any `INFO` logs.
transformers_logger.setLevel(logging.WARNING)

In [17]:

sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [ 5e-5, 3e-5, 2e-5]
        },
        'batch_size': {
            'values': [8, 16, 32]
        },
        'epochs':{
            'values':[2, 3, 4]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project='intent classifier')


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: h92if1xh
Sweep URL: https://wandb.ai/aescontrelar/intent%20classifier/sweeps/h92if1xh


In [20]:
import torch
torch.cuda.is_available()

True

In [21]:
wandb.login()

wandb: Currently logged in as: aescontrelar (use `wandb login --relogin` to force relogin)


True

In [22]:
    model_args = ClassificationArgs(num_train_epochs=3, use_early_stopping = True, max_seq_length=17)
    model = ClassificationModel(
        "bert",
        "bert-base-cased", num_labels=8, args=model_args,)

INFO:filelock:Lock 139806558410736 acquired on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

INFO:filelock:Lock 139806558410736 released on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock
INFO:filelock:Lock 139806254620512 acquired on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

INFO:filelock:Lock 139806254620512 released on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expec

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

INFO:filelock:Lock 139806735275736 released on /root/.cache/huggingface/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock


In [23]:
model.train_model(train_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/2052 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/257 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 3:   0%|          | 0/257 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/257 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


(771, 0.1945341752033352)

In [ ]:
#try wandb
def train():
    # Initialize a new wandb run
    wandb.init(project = "intent classifier")
    
    model_args = ClassificationArgs(num_train_epochs=3, use_early_stopping = True, max_seq_length=17)



    # Create a TransformerModel
    model = ClassificationModel(
        "bert",
        "bert-base-cased",

        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(
        train_df,
        eval_df=test_df,
     
        
    )

    # Sync wandb
    wandb.join()

if torch.cuda.is_available():
    device = 'cuda'
    wandb.agent(sweep_id, train)


INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None
wandb: Agent Starting Run: wgvlup34 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 4
wandb: 	learning_rate: 3e-05
wandb: Currently logged in as: aescontrelar (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep


INFO:filelock:Lock 139890252454880 acquired on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

INFO:filelock:Lock 139890252454880 released on /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock
INFO:filelock:Lock 139890252453480 acquired on /root/.cache/huggingface/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

Run wgvlup34 errored: RuntimeError('CUDA error: device-side assert triggered',)
wandb: ERROR Run wgvlup34 errored: RuntimeError('CUDA error: device-side assert triggered',)
wandb: Agent Starting Run: uosiu2yc with config:
wandb: 	batch_size: 16
wandb: 	epochs: 3
wandb: 	learning_rate: 3e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Run uosiu2yc errored: RuntimeError('CUDA error: device-side assert triggered',)
wandb: ERROR Run uosiu2yc errored: RuntimeError('CUDA error: device-side assert triggered',)
wandb: Agent Starting Run: ksc7p00i with config:
wandb: 	batch_size: 32
wandb: 	epochs: 3
wandb: 	learning_rate: 2e-05
wandb: WARNING Ignored wandb.init() arg project when running a sweep


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Run ksc7p00i errored: RuntimeError('CUDA error: device-side assert triggered',)
wandb: ERROR Run ksc7p00i errored: RuntimeError('CUDA error: device-side assert triggered',)
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true


In [ ]:
wandb.init()
model_args = ClassificationArgs(num_train_epochs=3, use_early_stopping = True, max_seq_length=17)

model = ClassificationModel('bert', 'bert-base-cased', num_labels= 8, args=model_args, sweep_config=wandb.config)
# Choose a "project name" to organize all of your "runs" underneath.
model.args.wandb_project = 'intent_classifier'

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
!rm -rf /content/outputs

In [24]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/513 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/65 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.9861757468157373, 'eval_loss': 0.06630752398005615}


In [26]:
result

{'eval_loss': 0.06630752398005615, 'mcc': 0.9861757468157373}

In [30]:
predictions, raw_outputs = model.predict(["when was published harry potter"])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
predictions

array([4])

In [32]:
encoder.inverse_transform([predictions])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['date'], dtype=object)

In [ ]:
import os
import tarfile

def pack_model(model_path='',file_name=''):
  files = [files for root, dirs, files in os.walk(model_path)][0]
  with tarfile.open(file_name+ '.tar.gz', 'w:gz') as f:
    for file in files:
      f.add(f'{model_path}/{file}')

# run the function
pack_model('outputs','model_intent')